In [ ]:
import h5py
import pandas as pd

from functools import reduce

In [ ]:
quotes = pd.read_hdf("data/quotes/quotes_pol_allpol1.h5")
elec = pd.read_parquet("data/elec/leg_shares.parquet")

## 1. Outcomes

In [ ]:
# Filter quotes to only include political alignments from elec dataframe
valid_alignments = elec['alignement_politique'].unique()
filtered_quotes = quotes[quotes['alignement_politique'].isin(valid_alignments)].copy()

# Convert date to datetime if it isn't already
if not pd.api.types.is_datetime64_any_dtype(filtered_quotes['date']):
    filtered_quotes['date'] = pd.to_datetime(filtered_quotes['date'])

# Extract month from date
filtered_quotes['month'] = filtered_quotes['date'].dt.to_period('M')

In [ ]:
# Number and share of quotes
quotes_counts = filtered_quotes.groupby(['month', 'journal', 'alignement_politique']).size().reset_index()
quotes_total_counts = filtered_quotes.groupby(['month', 'journal']).size().reset_index(name='quotes_nb')
quotes_data = pd.merge(quotes_counts, quotes_total_counts, on=['month', 'journal'])
quotes_data['quotes_share'] = quotes_data[0] / quotes_data['quotes_nb']
quotes_data.drop(columns=0, inplace=True)

In [ ]:
# Number and share of articles with quotes
art_counts = filtered_quotes.groupby(['month', 'journal', 'alignement_politique'])['text_id'].nunique().reset_index()
art_total_counts = filtered_quotes.groupby(['month', 'journal'])['text_id'].nunique().reset_index(name='art_nb')
art_data = pd.merge(art_counts, art_total_counts, on=['month', 'journal'])
art_data['art_share'] = art_data['text_id'] / art_data['art_nb']
art_data.drop(columns='text_id', inplace=True)

In [ ]:
# Quote length
filtered_quotes['quote_length'] = filtered_quotes['quote_text'].apply(lambda x: len(x.strip('« » ')))
length_data = filtered_quotes.groupby(['month', 'journal', 'alignement_politique'])['quote_length'].mean().reset_index(name='mean_quote_length')

In [ ]:
# Quote style
type_counts = filtered_quotes.groupby(['month', 'journal', 'alignement_politique', 'quote_type']).size().reset_index()
type_total_counts = filtered_quotes.groupby(['month', 'journal', 'alignement_politique']).size().reset_index(name='quotes_type_nb')
type_data = pd.merge(type_counts, type_total_counts, on=['month', 'journal', 'alignement_politique'])
type_data['type_quotes_share'] = type_data[0] / type_data['quotes_type_nb']

direct_data = (
    type_data[type_data['quote_type'] == 'Direct']
    .rename(columns={'type_quotes_share': 'type_direct_share'})
    .drop(columns=['quote_type', 'quotes_type_nb', 0]))

undirect_data = (
    type_data[type_data['quote_type'] == 'Indirect']
    .rename(columns={'type_quotes_share': 'type_undirect_share'})
    .drop(columns=['quote_type', 'quotes_type_nb', 0]))

mixed_data = (
    type_data[type_data['quote_type'] == 'Mixed']
    .rename(columns={'type_quotes_share': 'type_mixed_share'})
    .drop(columns=['quote_type', 'quotes_type_nb', 0]))

In [ ]:
# Personalities
person_data = filtered_quotes.groupby(['month', 'journal', 'alignement_politique'])['person_name_standardized'].nunique().reset_index(name='personalities_nb')

In [ ]:
# Final dataframe
dfs = [quotes_data, art_data, length_data, direct_data, undirect_data, mixed_data, person_data]
model_data = reduce(lambda left, right: pd.merge(left, right, on=['month', 'journal', 'alignement_politique']), dfs)

## 2. Identification des cycles électoraux

In [ ]:
legi_dates = [
    "21/06/1981",
    "16/03/1986",
    "11/06/1988",
    "28/03/1993",
    "01/06/1997",
    "16/06/2002",
    "17/06/2007",
    "17/06/2012",
    "18/06/2017",
    "19/06/2022",
    "06/07/2024"]

In [ ]:
# Remove entries older than the earliest election in the dataset
legi_dates = pd.to_datetime(legi_dates, format="%d/%m/%Y").to_period('M').sort_values()
model_data = model_data[model_data['month'] >= min(legi_dates)].copy().reset_index()

In [ ]:
# Find the year of the latest election
def find_last_election_year(current_month):
    relevant_elections = legi_dates[legi_dates <= current_month]

    if not relevant_elections.empty:
        return relevant_elections.max().year
    else:
        return pd.NA

model_data['last_election'] = model_data['month'].apply(find_last_election_year)
model_data['last_election'] = pd.to_datetime(model_data['last_election'], format="%Y")

## 3. Fusion avec les données des élections législatives

In [ ]:
elec = elec.groupby('alignement_politique', as_index=False).sum()
elec.drop(columns='candidat', inplace=True)
elec = elec.melt(id_vars=['alignement_politique'], var_name='last_election', value_name='votes_share')
elec['last_election'] = pd.to_datetime(elec['last_election'], format="%Y")

In [ ]:
model_data = pd.merge(model_data, elec, on=['alignement_politique', 'last_election'], how='left')
model_data.drop(columns=['last_election', 'index'], inplace=True)

## 4. Indicatrice de représentation au gouvernement
Comment traiter les cohabitations ? En l'état le bonus est seulement accordé à la nuance au gouvernement.

- **centre**
  - 19/06/2017 - 31/12/2024
- **centredroite_droite**
  - 20/03/1986 - 10/05/1988
  - 29/03/1993 - 02/06/1997
  - 06/05/2002 - 10/05/2012
- **centregauche_gauche**
  - 22/06/1981 - 20/03/1986
  - 10/05/1988 - 29/03/1993
  - 02/06/1997 - 06/05/2002
  - 15/05/2012 - 19/06/2017

In [ ]:
gov_periods = {
    "centre": [
        ("2017-06-19", "2024-12-31")
    ],
    "centredroite_droite": [
        ("1986-03-20", "1988-05-10"),
        ("1993-03-29", "1997-06-02"),
        ("2002-05-06", "2012-05-10"),
    ],
    "centregauche_gauche": [
        ("1981-06-22", "1986-03-20"),
        ("1988-05-10", "1993-03-29"),
        ("1997-06-02", "2002-05-06"),
        ("2012-05-15", "2017-06-19"),
    ],
    "extremedroite_droiteradicale": [],
    "extremegauche_gaucheradicale": [],
    "autre": [],
}

In [ ]:
all_start_dates = [pd.to_datetime(start) for periods in gov_periods.values() for start, _ in periods]
all_end_dates = [pd.to_datetime(end) for periods in gov_periods.values() for _, end in periods]
min_date = min(all_start_dates).replace(day=1)
max_date = max(all_end_dates).replace(day=1)
all_months = pd.date_range(start=min_date, end=max_date, freq="MS")

data = []

for alignment, periods in gov_periods.items():
    active_months = pd.Series(0, index=all_months)
    for start, end in periods:
        period_range = pd.date_range(
            start=pd.to_datetime(start).replace(day=1),
            end=pd.to_datetime(end).replace(day=1),
            freq="MS"
        )
        active_months.loc[period_range] = 1

    df = pd.DataFrame({
        "month": active_months.index.to_period('M'),
        "alignement_politique": alignment,
        "government": active_months.values
    })
    data.append(df)

gov_dummies = pd.concat(data)
gov_dummies.sort_values(by=["month", "alignement_politique"], inplace=True)
gov_dummies.reset_index(drop=True, inplace=True)

In [ ]:
model_data = pd.merge(model_data, gov_dummies, on=['alignement_politique', 'month'], how='left')
model_data.to_parquet(f"data/model_data.parquet", index=False)